In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

In [ ]:
## Simulate autoregressive process and calculate correlations and t statistic for different sample sizes

# Number of time iterations and different sample sizes
times = 100
ns = list(np.arange(50, 2000, 5))

# Create combinations of time and sample size
comb = pd.DataFrame(
    [(ix, n) for ix in range(1, times + 1) for n in ns],
    columns=['ix', 'n']
)
ncomb = len(comb)

# Initialize result array and column names
res = np.empty((ncomb, 5))
res[:] = np.nan
colnames = ['ix', 'n', 'cor', 'tstat', 'pval']

# Simulate an autoregressive process
def simulate_ar(n, phi):
    ar_process = [0]
    for _ in range(n):
        ar_process.append(phi * ar_process[-1] + np.random.normal())
    return ar_process[1:]

# Iterate through combinations and calculate correlations and statistics
for i in range(ncomb):
    n = comb.loc[i, 'n']
    ix = comb.loc[i, 'ix']
    sim1 = simulate_ar(n, phi=1)
    sim2 = simulate_ar(n, phi=1)
    cor, pval = pearsonr(sim1, sim2)
    tstat = cor * np.sqrt((n - 2) / (1 - cor**2))
    res[i, :] = [ix, n, cor, tstat, pval]

# Create a DataFrame from the results
tab = pd.DataFrame(res, columns=colnames)

# Group the DataFrame by 'n' and calculate summary statistics
tab_grouped = tab.groupby('n').agg(
    avg_abs_corr=('cor', lambda x: np.mean(np.abs(x))),
    avg_abs_tstat=('tstat', lambda x: np.mean(np.abs(x))),
    percent_sig=('pval', lambda x: np.mean(x < 0.05))
).reset_index()

# Round values for cleaner presentation
tab_grouped = tab_grouped.round(2)

# Print the grouped and summarized results
print(tab_grouped)
plt.figure()
plt.plot(tab_grouped['avg_abs_tstat'])
plt.savefig(r'C:\Users\Brandon\Desktop\PhD\Baseline Dynamics\Baseline-Dynamics\Figures\t_statistic_AR_process.pdf', format = 'pdf')
plt.figure()
plt.plot(tab_grouped['percent_sig'])
plt.savefig(r'C:\Users\Brandon\Desktop\PhD\Baseline Dynamics\Baseline-Dynamics\Figures\percent_significant_AR_process.pdf', format = 'pdf')
